In [1]:
import torch
import d2l as d2l
from torch import nn

In [5]:
def corr2d(X, k):
    """ 计算二维互相关运算 """
    h, w = k.shape
    Y = torch.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i:i + h, j:j + w] * k).sum()
    return Y

In [6]:
class Conv2D(nn.Module):
    """ 初始化卷积层 """
    def __init__(self, kernel_size):
        super().__init__()
        self.weight = nn.Parameter(torch.rand(kernel_size))
        self.bias = nn.Parameter(torch.zeros(1))

    def forward(self, x):
        return corr2d(x, self.weight) + self.bias

## 学习卷积核

In [9]:
conv2d = nn.Conv2d(1, 1, kernel_size=(1, 2), bias=False)     # 构造一个二维卷积层，具有一个输出通道和形状为（1,2）的卷积核
X = torch.ones((6, 8))
X[:, 2:6] = 0
K = torch.tensor([[1.0, -1.0]])
Y = corr2d(X, K)
X = X.reshape((1,1,6,8))
Y = Y.reshape((1,1,6,7))

for i in range(10):
    Y_hat = conv2d(X)
    l = (Y_hat-Y) ** 2
    conv2d.zero_grad()
    l.sum().backward()
    conv2d.weight.data[:] -= 3e-2 * conv2d.weight.grad
    if (i + 1) % 2 == 0:
        print(f'batch {i+1}, loss {l.sum():.3f}')

conv2d.weight.data.reshape((1, 2))

batch 2, loss 11.542
batch 4, loss 3.541
batch 6, loss 1.252
batch 8, loss 0.479
batch 10, loss 0.191


tensor([[ 1.0331, -0.9438]])